In [326]:
import numpy as np
from numpy.linalg import norm, eig, inv
from scipy.optimize import minimize
from scipy.linalg import sqrtm
from scipy.special import erf
import matplotlib.pyplot as plt
from itertools import product

In [147]:
class Gaussian():
    
    def __init__(self, exponent, atom=None):
        self.exponent = exponent
        self.atom = atom
        
    def overlap_integral(self, g):
        a = self.exponent
        b = g.exponent
        R_A = self.pos
        R_B = g.atom.pos

        return (4*a*b / (a + b) ** 2) ** (3/4) * np.exp(-a*b / (a + b) * np.linalg.norm(R_A - R_B)**2)
    
    def kinetic_integral(self, g, s=None):
        a = self.exponent
        b = g.exponent
        R_A = self.pos
        R_B = g.atom.pos

        c = a * b / (a + b)

        if s:
            return s * a * (3 - 2 * a * norm(R_A - R_B)^2)
        else:
            return overlap_integral(A, B) * c * (3 - 2 * c * np.linalg.norm(R_A - R_B)**2)
    
    @property
    def pos(self):
        return self.atom.pos

class Atom():
    
    def __init__(self, position, gaussians):
        self.pos = position
        self.gaussians = gaussians
        for g in self.gaussians:
            g.atom = self
    
class Molecule():
    
    def __init__(self, atoms): 
        self.atoms = atoms

In [280]:
# Integral functions
def overlap_integral(e1, p1, e2, p2):
    return (4*e1*e2 / (e1 + e2) ** 2) ** (3/4) * np.exp(-e1*e2 / (e1 + e2) * np.linalg.norm(p1 - p2)**2)


def overlap_matrix(pos, exponents):
    
    K = len(pos)*len(exponents)
    
    pairs = product(exponents, pos, repeat=2)
    
    S = [0] * K**2
    
    for i, a in enumerate(pairs):
        S[i] = overlap_integral(a[0], a[1], a[2], a[3])
            
    return np.reshape(S, (K, K))
                
    
    
def kinetic_integral(e1, p1, e2, p2, s=None):
    c = e1 * e2 / (e1 + e2)
    if s:
        return s * c * (3 - 2 * c * np.linalg.norm(p1 - p2)**2)
    else:
        return overlap_integral(e1, p1, e2, p2) * c * (3 - 2 * c * np.linalg.norm(p1 - p2)**2)
    
def kinetic_matrix(pos, exponents, S=None):
    K = len(pos)*len(exponents)
    pairs = product(exponents, pos, repeat=2)
    
    T = [0] * K**2
    flat_S = S.flatten()
    
    for i, a in enumerate(pairs):
        T[i] = kinetic_integral(a[0], a[1], a[2], a[3], flat_S[i])
            
    return np.reshape(T, (K, K))


def core_potential_integral(e1, p1, e2, p2, Z, pc, s=None):
    c = np.sqrt(e1 + e2)
    R_P = (e1 * p1 + e2 * p2) / (e1 + e2)
    
    if s:
        if (R_P == pc).all():
            return -s * Z * 2 * c / np.sqrt(np.pi)
        else:
            return -s * Z / norm(R_P - pc) * sp.special.erf(c * norm(R_P - pc))
    else:
        if (R_P == pc).all():
            return -overlap_integral(A, B) * Z * 2 * c / np.sqrt(np.pi)
        else:
            return -overlap_integral(A, B) * Z / norm(R_P - pc) * erf(c * norm(R_P - pc))
    
def core_potential_matrix(pos, exponents, S=None):
    K = len(pos)*len(exponents)
    pairs = product(exponents, pos, repeat=2)
    
    V = [0] * K**2
    flat_S = S.flatten()
    
    for i, a in enumerate(pairs):
        V[i] = sum([core_potential_integral(a[0], a[1], a[2], a[3], 1, p3, flat_S[i]) for p3 in pos])
            
    return np.reshape(V, (K, K))



# TODO: re-use overlap integrals
# Calculating a lot of overlap integrals everywhere...
def coulomb_integral(e1, p1, e2, p2, e3, p3, e4, p4, s12=None, s34=None):
    
    R_P = (e1 * p1 + e2 * p2) / (e1 + e2)
    R_Q = (e3 * p3 + e4 * p4) / (e3 + e4)
    
    c = np.sqrt((e1 + e2) * (e3 + e4) / (e1 + e2 + e3 + e4))
    
    if s12 and s34:
        if (R_P == R_Q).all():
            return s12 * s34 * 2 * c / np.sqrt(np.pi)
        else:
            return s12 * s34 / norm(R_P - R_Q) * erf(c * norm(R_P - R_Q))
    
    else:
        if (R_P == R_Q).all():
            return overlap_integral(e1, p1, e2, p2) * overlap_integral(e3, p3, e4, p4) * 2 * c / np.sqrt(np.pi)
        else:
            return overlap_integral(e1, p1, e2, p2) * overlap_integral(e3, p3, e4, p4) / norm(R_P - R_Q) * erf(c * norm(R_P - R_Q))
    
def coulomb_matrix(pos, exponents, S=None):
    K = len(pos)*len(exponents)
    pairs = product(exponents, pos, repeat=4)
    
    V = [0] * K**4
    flat_S = S.flatten()
    
    s_pairs = product(flat_S, repeat=2)
    
    for i, (a, s) in enumerate(zip(pairs, s_pairs)):
        V[i] = coulomb_integral(a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7], s[0], s[1])
            
    return np.reshape(V, (K, K, K, K))


def exchange_integral(e1, p1, e2, p2, e3, p3, e4, p4, s13=None, s24=None):
    return coulomb_integral(e1, p1, e3, p3, e2, p2, e4, p4, s13, s24) # Just change place

def exchange_matrix(pos, exponents, S=None):
    K = len(pos)*len(exponents)
    pairs = product(exponents, pos, repeat=4)
    
    M1 = [0] * K**4
    flat_S = S.flatten()
    
    # I need to make this more efficient
    #s_pairs = product(flat_S, repeat=2)
    
    for i, a in enumerate(pairs):
        M1[i] = exchange_integral(a[0], a[1], a[2], a[3], a[4], a[5], a[6], a[7])#, S[i,k], S[j,l])
            
    return np.reshape(M1,(K,K,K,K))



In [296]:
pos = np.array([[-1,0,0],[0,1,0],[1,0,0]])
#exponents = [0.3425250914e1, 0.6239137298, 0.1688554040]
exponents = [0.233136, 1.309757]

K = len(pos)*len(exponents)

#len(pos)
S = overlap_matrix(pos, exponents)
T = kinetic_integrals(pos, exponents, S)
V = core_potential_matrix(pos, exponents, S)
H = T + V

M0 = coulomb_matrix(pos, exponents, S)
M1 = exchange_matrix(pos, exponents, S)
M = M0 - 0.5*M1

M_square = np.reshape(M,(K**2,K**2))


P = np.ones((K,K))
G = np.reshape(M_square @ P.flatten(),(K,K))

F = H + G

In [406]:
def p_matrix(C,N):
    return  2 @ C[:,:N//2] @ C[:,:N//2].T

def hf_step(exponents, pos, N, C):
    K = len(pos)*len(exponents)
    print(K)
    S = overlap_matrix(pos, exponents)
    T = kinetic_integrals(pos, exponents, S)
    V = core_potential_matrix(pos, exponents, S)
    H = T + V

    M0 = coulomb_matrix(pos, exponents, S)
    M1 = exchange_matrix(pos, exponents, S)
    M = M0 - 0.5*M1

    M_square = np.reshape(M,(K**2,K**2))

    
    P = 2 * C[:,:N//2] * C[:,:N//2].T 
    
    print(M_square.shape)
    print(P.shape)
    G = np.reshape(M_square @ P.flatten(), (K,K))

    F = H + G
    
    X = sqrtm(inv(S))
    
    F_dash = X @ F @ X
    
    E, B = eig(F_dash) # Needs to be sorted
    E[E.argsort()]
    B[E.argsort()]
    C = X @ B
    
    return C, E[0]

def hartree_fock(exponents, pos, N, init_guess=None, tol=1e-6):
    C = init_guess
    diff = np.inf
    E = np.inf
    n = 1
    while diff > tol:
        P_old = p_matrix(C,N)
        C, E = hf_step(exponents, pos, N, C)
        P_new = p_matrix(C,N)
        diff = norm(P_new - P_old)
        n += 1
        
    return E

def geometry_optimization(exponents, pos, N, init_guess=None, tol=1e-6):
    def minimize_me(x):
        coords = np.reshape(pos,(3, len(x)//3))
        E = hartree_fock(exponents, coords, N, init_guess, tol)
        print(E, x)
        return E
    
    return minimize(minimize_me, pos, method='nelder-mead')

In [407]:
pos = np.array([[-1,0,0],[0,1,0]])#,[1,0,0]])
#exponents = [0.3425250914e1, 0.6239137298, 0.1688554040]
exponents = [0.233136, 1.309757]
K = len(pos)*len(exponents)
N = 2

E = hartree_fock(exponents, pos, N, init_guess=np.ones((K,K)), tol=1e-12)
print(E)

res = geometry_optimization(exponents, pos, N, init_guess=np.ones((6,6)))
rescoords

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [408]:
pos = np.array([[-1,0,0],[0,1,0]])#,[1,0,0]])
#exponents = [0.3425250914e1, 0.6239137298, 0.1688554040]
exponents = [0.233136, 1.309757]
K = len(pos)*len(exponents)
N = 2
init_guess = np.ones((K,K))

def minimize_me(x):
    coords = np.reshape(pos,(3, len(x)//3))
    E = hartree_fock(exponents, coords, N, init_guess)
    print(E, x)
    return E

x = np.linspace(0.3,2,100)
y = [minimize_me([0,0,0,i,0,0]) for i in x]

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)